In [ ]:
from typing import Optional
from functools import wraps
import traceback

class ECTError(Exception):
    def __init__(self, message: str):
        self.message = message
        super().__init__(self.message)

class DocumentNotFoundError(ECTError):
    pass

class ContentFetchError(ECTError):
    pass

class MetadataParsingError(ECTError):
    pass

class APIError(ECTError):
    def __init__(self, message: str, status_code: Optional[int] = None):
        self.status_code = status_code
        super().__init__(message)

def log_errors(logger):
    def decorator(func):
        @wraps(func)
        async def wrapper(*args, **kwargs):
            try:
                return await func(*args, **kwargs)
            except DocumentNotFoundError as e:
                logger.error("Document not found", error=e)
                raise
            except ContentFetchError as e:
                logger.error("Failed to fetch content", error=e)
                raise
            except MetadataParsingError as e:
                logger.error("Failed to parse metadata", error=e)
                raise
            except APIError as e:
                logger.error(f"API Error (status: {e.status_code})", error=e)
                raise
            except Exception as e:
                logger.error("Unexpected error", error=e)
                logger.debug(f"Traceback: {traceback.format_exc()}")
                raise ECTError(f"Unexpected error: {str(e)}")
        return wrapper
    return decorator